# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [97]:
import findspark
findspark.init('/home/shashank/spark-2.3.2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('exc').getOrCreate()
df = spark.read.csv('walmart_stock.csv', inferSchema=True, header=True)
df.createOrReplaceTempView('ex_sql')

#### Load the Walmart Stock CSV File, have Spark infer the data types.

#### What are the column names?

In [100]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [101]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [102]:
df.head(1)[0]

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

#### Use describe() to learn about the DataFrame.

In [103]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [104]:
a=df.describe()

In [105]:
a=a.withColumnRenamed('Adj Close', 'Adj_Close')

In [106]:
a.createOrReplaceTempView('a_sql')

In [107]:
a.dtypes #All string

[('summary', 'string'),
 ('Open', 'string'),
 ('High', 'string'),
 ('Low', 'string'),
 ('Close', 'string'),
 ('Volume', 'string'),
 ('Adj_Close', 'string')]

In [108]:
#Convert to double

b = spark.sql("FROM a_sql SELECT summary, CAST(Open AS DOUBLE) , CAST(High AS DOUBLE), CAST(Low AS DOUBLE), \
            CAST(Close AS DOUBLE), CAST(Volume AS DOUBLE), CAST(Adj_Close AS DOUBLE)")

In [109]:
b.dtypes #string and double

[('summary', 'string'),
 ('Open', 'double'),
 ('High', 'double'),
 ('Low', 'double'),
 ('Close', 'double'),
 ('Volume', 'double'),
 ('Adj_Close', 'double')]

In [110]:
b.createOrReplaceTempView('b_sql') #convert to sql dataframe

In [111]:
#format numbers
spark.sql("FROM b_sql SELECT summary, CAST(Open AS DECIMAL(15,2)) , CAST(High AS DECIMAL(15,2)), CAST(Low AS DECIMAL(15,2)), CAST(Close AS DECIMAL(15,2)), CAST(Volume AS DECIMAL(15,2)), CAST(Adj_Close AS DECIMAL(15,2))").show()

+-------+-------+-------+-------+-------+-----------+---------+
|summary|   Open|   High|    Low|  Close|     Volume|Adj_Close|
+-------+-------+-------+-------+-------+-----------+---------+
|  count|1258.00|1258.00|1258.00|1258.00|    1258.00|  1258.00|
|   mean|  72.36|  72.84|  71.92|  72.39| 8222093.48|    67.24|
| stddev|   6.77|   6.77|   6.74|   6.76| 4519780.84|     6.72|
|    min|  56.39|  57.06|  56.30|  56.42| 2094900.00|    50.36|
|    max|  90.80|  90.97|  89.25|  90.47|80898100.00|    84.91|
+-------+-------+-------+-------+-------+-----------+---------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [112]:
spark.sql("FROM ex_sql SELECT High/Volume AS HV_RATIO").show()

+--------------------+
|            HV_RATIO|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [113]:
df.head()

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

In [126]:
from pyspark.sql.functions import dayofmonth, month, year, mean
spark.sql("FROM ex_sql SELECT MAX(High) AS max_high").show()

+---------+
| max_high|
+---------+
|90.970001|
+---------+



In [122]:
spark.sql("FROM ex_sql SELECT DAY(Date), MONTH(Date), YEAR(Date) WHERE High == 90.970001").show()

+------------------------------+-------------------------+------------------------+
|dayofmonth(CAST(Date AS DATE))|month(CAST(Date AS DATE))|year(CAST(Date AS DATE))|
+------------------------------+-------------------------+------------------------+
|                            13|                        1|                    2015|
+------------------------------+-------------------------+------------------------+



#### What is the mean of the Close column?

In [131]:
spark.sql("FROM ex_sql SELECT AVG(Close)").show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [134]:
spark.sql("FROM ex_sql SELECT MIN(Volume), MAX(Volume)").show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [135]:
spark.sql("FROM ex_sql SELECT COUNT(Date) WHERE Close < 60").show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [139]:
spark.sql("FROM ex_sql SELECT COUNT(Date) WHERE High > 80").show()

+-----------+
|count(Date)|
+-----------+
|        115|
+-----------+



In [140]:
spark.sql("FROM ex_sql SELECT 115/COUNT(Date)").show()

+---------------------------------------------------+
|(CAST(115 AS DOUBLE) / CAST(count(Date) AS DOUBLE))|
+---------------------------------------------------+
|                                0.09141494435612083|
+---------------------------------------------------+



#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [142]:
from pyspark.sql.functions import corr
spark.sql("FROM ex_sql SELECT corr(High, Volume)").show()

+----------------------------------+
|corr(High, CAST(Volume AS DOUBLE))|
+----------------------------------+
|               -0.3384326061737161|
+----------------------------------+



#### What is the max High per year?

In [145]:
spark.sql("FROM ex_sql SELECT YEAR(Date) AS Year, MAX(High) GROUP BY YEAR(Date)").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [156]:
spark.sql("FROM ex_sql SELECT YEAR(Date) AS Year, MONTH(Date) AS Month, AVG(Close) GROUP BY YEAR(Date), MONTH(DATE) ORDER BY YEAR(Date), MONTH(Date)").show()

+----+-----+------------------+
|Year|Month|        avg(Close)|
+----+-----+------------------+
|2012|    1|        60.2354999|
|2012|    2|            60.898|
|2012|    3|60.433636818181796|
|2012|    4|60.149000150000006|
|2012|    5|61.456363409090905|
|2012|    6| 67.50380961904762|
|2012|    7| 72.40666661904763|
|2012|    8| 73.04478265217392|
|2012|    9| 74.18157921052631|
|2012|   10| 75.30619061904761|
|2012|   11| 71.10952333333333|
|2012|   12| 69.71100009999999|
|2013|    1| 69.09476142857143|
|2013|    2| 70.62315857894738|
|2013|    3| 73.43649940000002|
|2013|    4| 77.68954572727273|
|2013|    5| 77.81636368181817|
|2013|    6| 74.97800020000001|
|2013|    7| 77.11545418181818|
|2013|    8| 75.22409204545455|
+----+-----+------------------+
only showing top 20 rows



# Great Job!